# 5. Fine tuning

In [ ]:
!pip install -q transformers accelerate peft bitsandbytes trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 35.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.3/366.3 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install bitsandbytes

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !pip install -q transformers peft accelerate bitsandbytes trl datasets

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
The token `abcdddder` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticat

In [ ]:
# Import Core Libraries & Define Paths
import os
import json
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForCausalLM, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from datasets import Dataset
from tqdm.auto import tqdm # For progress bars

# --- Define Base Paths ---
project_base_path = '/content/drive/MyDrive/VLM_Assignment'
data_splits_path = os.path.join(project_base_path, 'splits', 'data_splits.json')
ground_truth_folder = os.path.join(project_base_path, 'ground_truth_cleaned')
raw_images_input_folder = os.path.join(project_base_path, 'images')

# Output directory for fine-tuned model
output_dir = os.path.join(project_base_path, 'gemma_finetuned_lora')
os.makedirs(output_dir, exist_ok=True)

print("Paths and core libraries loaded.")

Paths and core libraries loaded.


In [ ]:
# Cell 21: Load Base Model, Processor, and Tokenizer (Quantized)

model_id = "google/gemma-3-12b-it"

# Configure 4-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load the multimodal processor
# This processor handles both image and text tokenization/processing for Gemma 3
processor = AutoProcessor.from_pretrained(model_id)

# Load the model with quantization
# Use trust_remote_code=True for Gemma 3 as it's a custom architecture
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto", # Automatically map model to available devices (GPU)
    trust_remote_code=True
)

# Prepare model for k-bit training (important for LoRA with quantization)
model = prepare_model_for_kbit_training(model)

print("Gemma-3-12b-it model, processor, and tokenizer loaded and quantized.")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00005-of-00005.safetensors:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Gemma-3-12b-it model, processor, and tokenizer loaded and quantized.


In [ ]:
# Cell 22: Prepare Training Dataset

# Load the data splits JSON
with open(data_splits_path, 'r', encoding='utf-8') as f:
    data_splits = json.load(f)

train_data_info = data_splits['train']
print(f"Found {len(train_data_info)} samples in the training split.")

# Prepare data in a format suitable for SFTTrainer
# For Gemma 3 multimodal, the input is typically an image and a text prompt,
# and the target is the response. We'll format it as a chat.
# The SFTTrainer expects a 'text' column where each entry is the formatted chat.

formatted_train_data = []

# Define the instruction for the model
instruction = "Please extract and transcribe all the text you can see in this image. The text is in Gujarati language. Just provide the text content without any additional commentary"

print("Preparing training data...")
for item in tqdm(train_data_info, desc="Formatting training data"):
    image_filename = item['filename'] # e.g., Swarupsannidhan_0012.png
    image_path = os.path.join(raw_images_input_folder, image_filename) # Use raw images for training

    # Ground truth filename is same as image filename, but .txt
    gt_filename = os.path.splitext(image_filename)[0] + '.txt'
    gt_filepath = os.path.join(ground_truth_folder, gt_filename)

    if not os.path.exists(image_path):
        print(f"Warning: Image file not found for training: {image_path}. Skipping.")
        continue
    if not os.path.exists(gt_filepath):
        print(f"Warning: Ground truth file not found for training: {gt_filepath}. Skipping.")
        continue

    try:
        # Load the image
        image = Image.open(image_path).convert("RGB") # Ensure RGB for consistency
        # Read the ground truth text
        with open(gt_filepath, 'r', encoding='utf-8') as f:
            ground_truth_text = f.read().strip()

        # Create the chat messages format
        # The model will learn to complete the 'model' turn given the 'user' turn (image + text)
        messages = [
            {"role": "user", "content": [{"type": "image", "image": image}, {"type": "text", "text": instruction}]},
            {"role": "model", "content": [{"type": "text", "text": ground_truth_text}]}
        ]

        # Apply the processor's chat template to format the messages into a single string
        # This includes special tokens like <image>, <start_of_turn>, etc.
        # The `return_tensors="pt"` is for inference, but for SFTTrainer we need string output
        # For fine-tuning, we usually pass the raw messages and let the trainer/tokenizer handle it
        # However, for multimodal models, the processor handles the image tokenization.
        # SFTTrainer expects a 'text' column. We will use the processor to apply the chat template.
        # The processor will handle the image tokenization internally when applying the chat template.
        tokenized_chat = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)
        formatted_train_data.append({"text": tokenized_chat})

    except Exception as e:
        print(f"Error processing {image_filename} for training: {e}. Skipping.")
        continue

# Create a Hugging Face Dataset
train_dataset = Dataset.from_list(formatted_train_data)
print(f"Prepared training dataset with {len(train_dataset)} samples.")

# Optional: Print a sample to verify format
# print("\nSample formatted training data entry:")
# print(train_dataset[0]['text'])

Found 70 samples in the training split.
Preparing training data...


Formatting training data:   0%|          | 0/70 [00:00<?, ?it/s]

Prepared training dataset with 70 samples.


In [ ]:
print("\nSample formatted training data entry:")
print(train_dataset[0]['text'])


Sample formatted training data entry:
<bos><start_of_turn>user
<start_of_image>Please extract and transcribe all the text you can see in this image. The text is in Gujarati language. Just provide the text content without any additional commentary<end_of_turn>
<start_of_turn>model
સ્વરૂપ સન્નિઘાન ૦ ૬
રચનાની મૂળ (ટૅક્સ્ટ એકલી નહોતી, બલકે તેમાં ભાષાનાં સૂર આદિ
તત્ત્વોનો સમાસ હતો, સંગીતનાં ગાયન-વાદન વાનાંનોય સમાસ હતો.
એ ટોટલ, પર્ફોર્મન્સ, ટૅક્સ્ટ હતી, એવો એ ડીસ્કોર્સ પોતે, ટૅક્સ્ટ
હતી. સંશોધન-પ્રકાશનમાં આ જીવન્ત સંક્રમણના કોઈ નિશાન ન
હોય, આ ટોટલ પર્ફોર્મન્સના હાડપિંજર જેવી ભાષા હોય ખરી. એ
ખોટ મોટી છે, ને ઉપેક્ષા કરવા જેવી તુચ્છ નથી.
ત્રીજી વાત એ છે કે મઘ્યકાલીન સાહિત્યમાં ગણાતી રચનાનો
રચયિતા આજના જેવો સર્જક નહોતો. આજના સાહિત્યકારની જેમ
એણે સભાનપણે સસંકલ્પ કશું સાહિત્ય ઊભું કરવાનું નહોતું કર્યુ.
એમની એવી, કારકિર્દી નહોતી. “એ બધા કણ્ઠોપકણ્ઠ પરમ્પરાના
વત્તેઓછે સ્વરૂપે અનુયાયીઓ હતા. પર્ફોર્મન્સ એમનાં પોતાનાં ખરાં,
ને એ જ એમની ખ્યાતિનાં મૂળાધારુ પણ રચનાના ઢાંચાઓ,
રચનાની ભાતો, તેનાં વલણો, તેની 

In [ ]:
# Cell 23: Configure LoRA

lora_config = LoraConfig(
    r=16, # Rank of the update matrices. Common values: 8, 16, 32, 64
    lora_alpha=32, # Scaling factor for LoRA updates
    target_modules="all-linear", # Apply LoRA to all linear layers (common for LLMs)
    lora_dropout=0.05, # Dropout probability for LoRA layers
    bias="none", # Don't fine-tune biases
    task_type="CAUSAL_LM", # This is a causal language modeling task
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Print trainable parameters to verify LoRA setup
model.print_trainable_parameters()

print("LoRA configured and applied to the model.")

trainable params: 74,165,760 || all params: 12,261,490,800 || trainable%: 0.6049
LoRA configured and applied to the model.


In [ ]:
# Cell 24: Configure Training Arguments

from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=1, # Start with 1 due to VRAM constraints with 12B model
    gradient_accumulation_steps=4, # Accumulate gradients over 4 steps to simulate a larger batch size of 4
    learning_rate=2e-4, # Common learning rate for LoRA
    num_train_epochs=3, # Number of training epochs. Adjust based on convergence.
    logging_steps=10, # Log training progress every N steps
    save_steps=100, # Save checkpoint every N steps
    save_total_limit=2, # Only keep the last 2 checkpoints
    fp16=False, # Already using bfloat16 via bnb_config
    bf16=True, # Enable bfloat16 training
    optim="paged_adamw_8bit", # Optimized AdamW for 8-bit quantization
    report_to="none", # Disable reporting to W&B etc. for simplicity
    remove_unused_columns=False, # Important for multimodal inputs with SFTTrainer
)

print("Training arguments configured.")

Training arguments configured.


In [ ]:
# Cell 25: Create and Run Trainer

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    peft_config=lora_config,
)

print("Starting fine-tuning...")
trainer.train()
print("\nFine-tuning complete!")

Converting train dataset to ChatML:   0%|          | 0/70 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/70 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/70 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/70 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting fine-tuning...


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,12.239300
20,9.916500
30,9.466700
40,8.481300
50,8.016500



Fine-tuning complete!


In [ ]:
# Cell 26: Save Fine-tuned LoRA Adapters

# Save the LoRA adapters
trainer.save_model(output_dir)

# To save the full model (base + merged LoRA adapters), you would do:
# merged_model = model.merge_and_unload()
# merged_model.save_pretrained(os.path.join(output_dir, "merged_model"))
# processor.save_pretrained(os.path.join(output_dir, "merged_model"))

print(f"Fine-tuned LoRA adapters saved to: {output_dir}")

Fine-tuned LoRA adapters saved to: /content/drive/MyDrive/VLM_Assignment/gemma_finetuned_lora


In [ ]:
# Cell 27: Load Fine-tuned Model and Processor for Inference
import os
import torch
from transformers import AutoProcessor, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
from PIL import Image

# Define the paths (make sure these match your setup)
project_base_path = '/content/drive/MyDrive/VLM_Assignment'
finetuned_model_path = os.path.join(project_base_path, 'gemma_finetuned_lora') # This is the output_dir from training
model_id = "google/gemma-3-12b-it" # Original base model ID

# Configure 4-bit quantization (should match training config for consistency)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f"Loading base model '{model_id}' in 4-bit...")
# Load the base model (in quantized form)
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True # Crucial for Gemma 3
)

print(f"Loading LoRA adapters from '{finetuned_model_path}'...")
# Load the LoRA adapters
model = PeftModel.from_pretrained(base_model, finetuned_model_path)

# Merge LoRA adapters into the base model for cleaner inference (optional but recommended)
# This will make 'model' a regular transformers model with merged weights
print("Merging LoRA adapters into the base model...")
model = model.merge_and_unload()
model.eval() # Set model to evaluation mode

# Load the processor (tokenizer + image processor)
processor = AutoProcessor.from_pretrained(model_id)

print("Fine-tuned model and processor loaded for inference.")

Loading base model 'google/gemma-3-12b-it' in 4-bit...


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Loading LoRA adapters from '/content/drive/MyDrive/VLM_Assignment/gemma_finetuned_lora'...
Merging LoRA adapters into the base model...


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/bnb.py:351: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(


Fine-tuned model and processor loaded for inference.
